In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/MyDrive/Colab Notebooks'

/content/drive/MyDrive/Colab Notebooks


In [ ]:

import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Lambda
from tensorflow.keras.models import Model
from keras import losses

from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import RootMeanSquaredError

In [ ]:
import pandas as pd
import numpy as np
import string
import os
import re
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
movies = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/CAERS/movie.csv")
rating = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/CAERS/rating.csv")
users  = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/CAERS/user.csv")

In [ ]:
movies.reset_index(inplace=True)
movies.rename(columns={'index':'movie_id'},inplace=True)
movies.head(2)

,movie_id,movie_name,genre,wikipedia_page_name,movie_plot
0,1,Toy Story (1995),"[Animation, Children's, Comedy]",Toy Story,A group of living toys are preparing to move i...
1,2,Jumanji (1995),"[Adventure, Children's, Fantasy]",Jumanji,"In 1969, Alan Parrish lives with his parents, ..."


In [ ]:
allData = rating.merge(movies, on='movie_id', how='inner')
allData = allData.merge(users, on='user_id', how='inner')

In [ ]:
allData.dropna(inplace=True)
allData.shape

(935815, 12)

In [ ]:
allData = allData[allData['movie_plot'].apply(len) > 0]
allData.shape

(929231, 12)

In [ ]:
allData['movie_id'].duplicated().sum()


926130

In [ ]:
allData['movie_id'] = pd.factorize(allData['movie_id'])[0] + 1
allData['movie_id'].duplicated().sum()

926130

In [ ]:
movies = allData[['movie_plot', 'genre', 'movie_id','movie_name']]

# Remove duplicates in the 'movies' DataFrame
movies.drop_duplicates(subset=['movie_plot', 'movie_id'], inplace=True)


<ipython-input-14-04feedbc6d20>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies.drop_duplicates(subset=['movie_plot', 'movie_id'], inplace=True)


In [ ]:
movies.shape

(3101, 4)

# Movie Embeding

##Stemming Lematizing

In [ ]:
# remove punctuation
movies['movie_plot'] = movies['movie_plot'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

<ipython-input-16-3f46fdef91b3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['movie_plot'] = movies['movie_plot'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))


In [ ]:
df_movies = pd.DataFrame()
movieEmbeding = pd.DataFrame()
df_movies['movie_plot']  = movies['movie_plot'].apply(lambda x: word_tokenize(x.lower()))
df_movies['movie_name'] = movies['movie_name'].apply(lambda x: word_tokenize(x.lower()))
#Stop words
stop_words = set(stopwords.words('english'))
df_movies['movie_plot'] = df_movies['movie_plot'].apply(lambda sentence: [word for word in sentence if word not in stop_words])

#Lemmatizer
lemmatizer = WordNetLemmatizer()
df_movies['movie_plot'] = df_movies['movie_plot'].apply(lambda sentence: [lemmatizer.lemmatize(word) for word in sentence])

#stemming
stemmer = PorterStemmer()
df_movies['movie_plot'] = df_movies['movie_plot'].apply(lambda sentence: [stemmer.stem(word) for word in sentence])


genre_model = Word2Vec(sentences=movies['genre'], vector_size=150, window=5, min_count=1, sg=0)
movieplot_model = Word2Vec(sentences=df_movies['movie_plot'], vector_size=150, window=5, min_count=1, sg=0)
movie_name_model = Word2Vec(sentences=df_movies['movie_name'], vector_size=150, window=5, min_count=1, sg=0)
movieEmbeding['genre'] = movies['genre'].apply(lambda x: sum(genre_model.wv[word] for word in x) / len(x) if len(x) > 0 else [])
movieEmbeding['plot'] = df_movies['movie_plot'].apply(lambda x: sum(movieplot_model.wv[word] for word in x) / len(x) if len(x) > 0 else [])
movieEmbeding['name'] = df_movies['movie_name'].apply(lambda x: sum(movie_name_model.wv[word] for word in x) / len(x) if len(x) > 0 else [])

In [ ]:
movieEmbeding.head()

,genre,plot,name
0,"[-0.0013995122, -0.0019394505, 0.0033830164, 0...","[-0.016021261, -0.2629913, -0.15489519, -0.048...","[0.053033512, -0.09657737, 0.006098273, 0.0335..."
1,"[-0.0022986748, -0.007900165, -0.0013379814, -...","[0.027112337, -0.30341187, -0.15935002, -0.040...","[0.06604706, -0.123547345, 0.008158322, 0.0413..."
2,"[0.0049209166, -0.006579022, -0.0024316232, 0....","[0.0032427951, -0.30434188, -0.09374804, -0.08...","[0.063571855, -0.11086556, 0.009079414, 0.0401..."
4,"[-0.0028716077, -0.008093305, 3.374713e-05, -0...","[-0.003315974, -0.2541745, -0.15427758, -0.067...","[0.0801369, -0.14743635, 0.0083915945, 0.05188..."
6,"[-0.0047036903, -0.0024089587, -0.0005393051, ...","[-0.007301961, -0.25236598, -0.13461363, -0.09...","[0.07179365, -0.13265258, 0.007773872, 0.04617..."


In [ ]:
movieEmbeding.isna().sum()

genre    0
plot     0
name     0
dtype: int64

In [ ]:
movieEmbeding.shape

(3101, 3)

#User Embeding

In [ ]:
users = allData[['user_id','age'	,'gender'	,'occupation'	,'zip_code']]

In [ ]:
users = users.drop_duplicates()

In [ ]:
users.shape

(6040, 5)

In [ ]:
users.tail()

,user_id,age,gender,occupation,zip_code
1000035,3537,1,M,K-12 student,97402
1000092,2908,18,M,artist,74403
1000114,2982,25,M,college/grad student,54902
1000135,3893,25,M,doctor/health care,79401
1000185,4211,45,M,customer service,77662


In [ ]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import pandas as pd
from sklearn.preprocessing import MinMaxScaler


userEmbeddings = pd.DataFrame()

# Preprocess and tokenize the "occupation" column
userEmbeddings['occupation'] = users['occupation'].apply(lambda x: word_tokenize(x.lower()))

# Concatenate the two lists within each row, handling NaN values
# userEmbeddings['combined_occupation'] = userEmbeddings['occupation'].apply(lambda x: x[0] + x[1] if len(x) == 2 else x[0] if len(x) == 1 else [])
# Train Word2Vec model for occupation
model_occ = Word2Vec(sentences=users['occupation'], vector_size=150, window=5, min_count=1, sg=0)

# Train Word2Vec model for gender
model_gen = Word2Vec(sentences=users['gender'].apply(lambda x: [x]), vector_size=150, window=5, min_count=1, sg=0)

# Create userEmbeddings DataFrame

# Use Word2Vec embeddings for occupation and gender
userEmbeddings['occupation'] = users['occupation'].apply(lambda x: sum(model_occ.wv[word] for word in x) / len(x) if len(x) > 0 else [])
userEmbeddings['gender'] = users['gender'].apply(lambda x: model_gen.wv[x])




In [ ]:
tokenized_zip_codes = [zip_code.split() for zip_code in users['zip_code']]
model_zip = Word2Vec(sentences=tokenized_zip_codes, vector_size=150, window=5, min_count=1, sg=0)
userEmbeddings['zip_code'] = [model_zip.wv[i] for i in tokenized_zip_codes]

In [ ]:
userEmbeddings['zip_code'] = userEmbeddings['zip_code'].apply(lambda x: x[0])

In [ ]:
userEmbeddings.head()

,occupation,gender,zip_code
0,"[-0.08244082, -0.0008182538, 0.019372748, -0.0...","[0.005211714, -0.0063401246, -0.00013702075, 0...","[-0.0056587444, -0.00037694056, -0.0020437622,..."
53,"[0.016136553, -0.023974879, -0.04114212, 0.140...","[-0.00035748482, 0.0001576209, 0.0034022331, 0...","[-0.0005099368, -0.0054536103, -0.0038579528, ..."
182,"[-0.083986856, -0.21259555, -0.023292294, 0.21...","[-0.00035748482, 0.0001576209, 0.0034022331, 0...","[-0.0066545517, -0.005130829, 0.0019386458, 0...."
205,"[-0.22619554, -0.3902278, 0.012600623, 0.10188...","[-0.00035748482, 0.0001576209, 0.0034022331, 0...","[0.005211714, -0.0063401246, -0.00013702075, 0..."
406,"[-0.20163473, -0.3323854, 0.06221606, 0.098973...","[-0.00035748482, 0.0001576209, 0.0034022331, 0...","[-0.0013801837, 0.005640979, -0.0031959882, -0..."


In [ ]:
movieEmbedingNp =  movieEmbeding.to_numpy()
userEmbeddings.dropna(inplace=True)
userEmbeddingNp =  userEmbeddings.to_numpy()


In [ ]:
movieEmbeding.head()

,genre,plot,name
0,"[-0.0013995122, -0.0019394505, 0.0033830164, 0...","[-0.016021261, -0.2629913, -0.15489519, -0.048...","[0.053033512, -0.09657737, 0.006098273, 0.0335..."
1,"[-0.0022986748, -0.007900165, -0.0013379814, -...","[0.027112337, -0.30341187, -0.15935002, -0.040...","[0.06604706, -0.123547345, 0.008158322, 0.0413..."
2,"[0.0049209166, -0.006579022, -0.0024316232, 0....","[0.0032427951, -0.30434188, -0.09374804, -0.08...","[0.063571855, -0.11086556, 0.009079414, 0.0401..."
4,"[-0.0028716077, -0.008093305, 3.374713e-05, -0...","[-0.003315974, -0.2541745, -0.15427758, -0.067...","[0.0801369, -0.14743635, 0.0083915945, 0.05188..."
6,"[-0.0047036903, -0.0024089587, -0.0005393051, ...","[-0.007301961, -0.25236598, -0.13461363, -0.09...","[0.07179365, -0.13265258, 0.007773872, 0.04617..."


In [ ]:
U = np.array([np.concatenate(col) for col in userEmbeddingNp])
M = np.array([np.concatenate(col) for col in movieEmbedingNp])


In [ ]:
U.shape

(6040, 450)

In [ ]:
M.shape

(3101, 450)

In [ ]:

dot = np.dot(U, M.T)

#Label

In [ ]:
# label_data = allData[(allData['user_id']<=2000) & (allData['movie_id']<=2000) ]

In [ ]:
labels = allData.pivot(index='user_id', columns='movie_id', values='rating')

In [ ]:
sum(labels.isna())

4809651

In [ ]:
labels = labels.fillna(0)
# labels = labels.drop(labels.index[-1])

In [ ]:
labels.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,3092,3093,3094,3095,3096,3097,3098,3099,3100,3101
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,3.0,5.0,5.0,5.0,4.0,4.0,5.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
pd.DataFrame(dot).to_pickle('DOT.pkl')
pd.DataFrame(labels).to_pickle('LABELS.pkl')

In [ ]:
dot = pd.read_pickle('DOT.pkl')
labels = pd.read_pickle('LABELS.pkl')